In [ ]:
from google.colab import drive
drive.mount("/drive")
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LinearRegression
sns.set()

Mounted at /drive


In [ ]:
df= pd.read_csv("/drive/My Drive/united1.csv",thousands=',',skipinitialspace=True)
df.head()

,DiaChi,TinhTrangBDS,DienTich,Gia/m2,Phongngu,TenPhanKhu,SoTang,PhongTam,Loai,GiayTo,MaCanHo,TinhTrangNoiThat,HuongCuaChinh,HuongBanCong,DacDiem,Gia
0,"Đường Liên Phường, Phường Phú Hữu, Quận 9, Tp ...",Đã bàn giao,50 m²,30 triệu/m²,2 phòng,NaN,NaN,1 phòng,Chung cư,Đã có sổ,NaN,Hoàn thiện cơ bản,Đông Nam,Đông,NaN,"1,5 tỷ- 50 m2đ"
1,"Đường Nguyễn Xiển, Phường Long Thạnh Mỹ, Quận ...",Đã bàn giao,34 m²,"52,35 triệu/m²",2 phòng,NaN,NaN,NaN,Chung cư,Đã có sổ,NaN,NaN,NaN,NaN,NaN,"1,78 tỷ- 34 m2đ"
2,"Lô 5, Lô 5 Khu đô thị Vĩnh Lộc, Phường Bình Hư...",Đã bàn giao,63 m²,"26,19 triệu/m²",2 phòng,NaN,NaN,2 phòng,Chung cư,Đang chờ sổ,NaN,Hoàn thiện cơ bản,NaN,NaN,NaN,"1,65 tỷ- 63 m2đ"
3,Tô Thị Hồng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,17 tỷ- 59 m2đ"
4,"Đường Liên Phường, Phường Phú Hữu, Quận 9, Tp ...",Đã bàn giao,53 m²,"30,19 triệu/m²",2 phòng,NaN,NaN,1 phòng,Chung cư,Đã có sổ,NaN,NaN,NaN,NaN,NaN,"1,6 tỷ- 53 m2đ"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29731 entries, 0 to 29730
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DiaChi            29731 non-null  object 
 1   TinhTrangBDS      27048 non-null  object 
 2   DienTich          27040 non-null  object 
 3   Gia/m2            27039 non-null  object 
 4   Phongngu          27050 non-null  object 
 5   TenPhanKhu        7542 non-null   object 
 6   SoTang            7220 non-null   float64
 7   PhongTam          26441 non-null  object 
 8   Loai              27050 non-null  object 
 9   GiayTo            20537 non-null  object 
 10  MaCanHo           3518 non-null   object 
 11  TinhTrangNoiThat  13798 non-null  object 
 12  HuongCuaChinh     10090 non-null  object 
 13  HuongBanCong      9353 non-null   object 
 14  DacDiem           5968 non-null   object 
 15  Gia               29731 non-null  object 
dtypes: float64(1), object(15)
memory usage: 

In [ ]:
## Mình sẽ chỉ lấy những data có dữ liệu về diện tích 
df1 = df[df.DienTich.notnull()]
df1 = df1.reset_index()
for i in range(0,len(df1.DienTich)):
    df1.DienTich[i]= df1.DienTich[i].split(' ')[0]
## Có thể thấy 2 trường dữ liệu Gia/m2 và Gia tương đương nhau nên mình sẽ xóa trường này đi
del df1["Gia/m2"]

In [ ]:
## Mình sẽ chỉ lấy những data có dữ liệu về phòng ngủ 
df2 = df1[df1.Phongngu.notnull()]
for i in range(0,len(df2.Phongngu)):
    if 'nhiều hơn' in df2.Phongngu[i]:
        df2.Phongngu[i]= float(df2.Phongngu[i].split(' ')[2])+1
    else:
        df2.Phongngu[i]= df2.Phongngu[i].split(' ')[0]

In [ ]:

## Mình sẽ chỉ lấy những data có dữ liệu về phòng tắm
df3 = df2[df2.PhongTam.notnull()]
df3 = df3.reset_index()
for i in range(0,len(df3.PhongTam)):
    if 'Nhiều hơn' in df3.PhongTam[i]:
        df3.PhongTam[i]= float(df3.PhongTam[i].split(' ')[2])+1
    else:
        df3.PhongTam[i]= df3.PhongTam[i].split(' ')[0]
        
del df3['level_0']
del df3['index']
df4 = df3[df3.Loai.notnull()]

In [ ]:
## truc quan truong gia tri Gia
for i in range(0,len(df4.Gia)):
    if 'GIÁ TỐT' in df4.Gia[i]:
        df4.Gia[i] = df4.Gia[i].split('\n')[0]
    else:
        df4.Gia[i] = df4.Gia[i].split('-')[0]
    

In [ ]:
#chuyen string thanh gia tri thuc
for i in range(0,len(df4.Gia)):
    if 'tỷ' in df4.Gia[i]:
        price = df4.Gia[i].split(' ')[0]
        price = price.replace(',','.')
        df4.Gia[i] = round(float(price)*1000000000,1)
    elif 'triệu' in df4.Gia[i] :
        price = df4.Gia[i].split(' ')[0]
        price = price.replace(',','.')
        df4.Gia[i] = round(float(price)*1000000,1)
    
df4 = df4.drop([17284])
df4 = df4.reset_index()
#for i in range(0,len(df4.Gia)):
#print df4



In [ ]:
df4['Quan'] = df.DiaChi.copy()
for i in range(0,len(df4.DiaChi)):
    a= df4.DiaChi[i].split(',')
    df4['Quan'][i] = a[len(a)-2]
for i in df4['Quan']:
    if 'Quận' not in i and 'Huyện' not in i:
        df4 = df4.drop(df4[df4['Quan']==i].index)

In [ ]:
df4.Quan.unique()

array([' Quận 9', ' Quận Bình Tân', ' Quận Thủ Đức', ' Quận 12',
       ' Quận 8', ' Quận Bình Thạnh', ' Huyện Bình Chánh', ' Quận 7',
       ' Quận Tân Bình', ' Huyện Nhà Bè', ' Quận Tân Phú', ' Quận 6',
       ' Quận 2', ' Quận 4', ' Quận Gò Vấp', ' Quận 5', ' Quận 1',
       ' Quận Phú Nhuận', ' Huyện Hóc Môn', ' Quận 10', ' Quận 3',
       ' Quận 11', ' Huyện Củ Chi'], dtype=object)

In [ ]:
frame = df4[['DiaChi','Quan','TinhTrangBDS','DienTich','Phongngu','TenPhanKhu','SoTang','PhongTam','Loai','GiayTo','MaCanHo'
             ,'TinhTrangNoiThat' ,'HuongCuaChinh','HuongBanCong','DacDiem',"Gia"]]
frame.head(2)

,DiaChi,Quan,TinhTrangBDS,DienTich,Phongngu,TenPhanKhu,SoTang,PhongTam,Loai,GiayTo,MaCanHo,TinhTrangNoiThat,HuongCuaChinh,HuongBanCong,DacDiem,Gia
0,"Đường Liên Phường, Phường Phú Hữu, Quận 9, Tp ...",Quận 9,Đã bàn giao,50,2,NaN,NaN,1,Chung cư,Đã có sổ,NaN,Hoàn thiện cơ bản,Đông Nam,Đông,NaN,1.5e+09
1,"Lô 5, Lô 5 Khu đô thị Vĩnh Lộc, Phường Bình Hư...",Quận Bình Tân,Đã bàn giao,63,2,NaN,NaN,2,Chung cư,Đang chờ sổ,NaN,Hoàn thiện cơ bản,NaN,NaN,NaN,1.65e+09


In [ ]:
frame.DienTich = frame.DienTich.astype('float')
frame.Phongngu = frame.Phongngu.astype('float') 
frame.PhongTam = frame.PhongTam.astype('float') 
frame.Gia = frame.Gia.astype('float') 
frame.describe()

,DienTich,Phongngu,SoTang,PhongTam,Gia
count,26136.000000,26136.000000,7039.000000,26136.000000,2.613600e+04
mean,218.358106,2.027127,11.330587,1.731558,2.756281e+09
std,9545.739259,0.741453,14.896840,0.610244,7.593491e+09
min,1.000000,1.000000,1.000000,1.000000,1.000000e+06
25%,55.000000,2.000000,5.000000,1.000000,1.600000e+09
50%,66.660000,2.000000,9.000000,2.000000,2.100000e+09
75%,78.000000,2.000000,16.000000,2.000000,3.150000e+09
max,1000000.000000,11.000000,720.000000,7.000000,9.800000e+11


In [ ]:
frame['USD'] = round(frame['Gia']/24000,0)
rows  = frame[frame.DienTich > 500]
frame = frame.drop(index = rows.index)
rows  = frame[frame.SoTang > 81]
frame = frame.drop(index = rows.index)
frame['log_price'] = np.log(frame.USD)

print("The number of row after cleaning data:",len(frame))

The number of row after cleaning data: 26068


In [ ]:
frame.to_csv('/drive/My Drive/dataclean123.csv',encoding="utf-8-sig",index=False)

In [ ]:
#thuat toan keras
df = pd.read_csv('/drive/My Drive/dataclean123.csv')

In [ ]:
df.describe()

,DienTich,Phongngu,SoTang,PhongTam,Gia,USD,log_price
count,26068.000000,26068.000000,7016.000000,26068.000000,2.606800e+04,2.606800e+04,26068.000000
mean,70.399262,2.024896,10.995867,1.729553,2.727795e+09,1.136581e+05,11.310788
std,30.291510,0.735334,7.351852,0.604371,7.487127e+09,3.119636e+05,1.004933
min,1.000000,1.000000,1.000000,1.000000,1.000000e+06,4.200000e+01,3.737670
25%,55.000000,2.000000,5.000000,1.000000,1.600000e+09,6.666700e+04,11.107465
50%,66.000000,2.000000,9.000000,2.000000,2.100000e+09,8.750000e+04,11.379394
75%,78.000000,2.000000,16.000000,2.000000,3.150000e+09,1.312500e+05,11.784859
max,500.000000,11.000000,70.000000,7.000000,9.800000e+11,4.083333e+07,17.525009


In [ ]:
df['Rank'] = df.USD.copy()
df_1 = df[df['USD'] < 50000]
df_2 = df[(df['USD'] >= 50000) & (df['USD'] < 100000)]
df_3 = df[(df['USD'] >= 100000) & (df['USD'] < 150000)]
df_4 = df[df['USD'] >= 150000]

In [ ]:
df_2.head(2)

,DiaChi,Quan,TinhTrangBDS,DienTich,Phongngu,TenPhanKhu,SoTang,PhongTam,Loai,GiayTo,MaCanHo,TinhTrangNoiThat,HuongCuaChinh,HuongBanCong,DacDiem,Gia,USD,log_price,Rank
0,"Đường Liên Phường, Phường Phú Hữu, Quận 9, Tp ...",Quận 9,Đã bàn giao,50.0,2.0,NaN,NaN,1.0,Chung cư,Đã có sổ,NaN,Hoàn thiện cơ bản,Đông Nam,Đông,NaN,1.500000e+09,62500.0,11.042922,62500.0
1,"Lô 5, Lô 5 Khu đô thị Vĩnh Lộc, Phường Bình Hư...",Quận Bình Tân,Đã bàn giao,63.0,2.0,NaN,NaN,2.0,Chung cư,Đang chờ sổ,NaN,Hoàn thiện cơ bản,NaN,NaN,NaN,1.650000e+09,68750.0,11.138232,68750.0


In [ ]:
## Tìm những trường dữ liệu có giá trị NaN
nan_values = df.isna()
nan_columns = nan_values.any()
columns_with_nan = df.columns[nan_columns].tolist()
columns_with_nan

['TinhTrangBDS',
 'TenPhanKhu',
 'SoTang',
 'GiayTo',
 'MaCanHo',
 'TinhTrangNoiThat',
 'HuongCuaChinh',
 'HuongBanCong',
 'DacDiem']

In [ ]:
## Tỉ lệ NaN của các trường dữ liệu
df.isna().sum()/df.shape[0]*100

DiaChi               0.000000
Quan                 0.000000
TinhTrangBDS         0.007672
DienTich             0.000000
Phongngu             0.000000
TenPhanKhu          71.743133
SoTang              73.085776
PhongTam             0.000000
Loai                 0.000000
GiayTo              23.588308
MaCanHo             86.815252
TinhTrangNoiThat    48.553782
HuongCuaChinh       62.095289
HuongBanCong        64.788246
DacDiem             77.777352
Gia                  0.000000
USD                  0.000000
log_price            0.000000
Rank                 0.000000
dtype: float64

In [ ]:
per =0.9 # Chọn xóa những cột dữ liệu có trên 50% data là NaN

In [ ]:
df_dropped = df.dropna(axis=1,thresh=int(df.shape[0]*per))
df_dropped_2 = df_dropped.dropna(how='any')
#df_dropped_2 = df_dropped.dropna(axis=0,thresh=int(df.shape[1]*0.5))
df_dropped_2

,DiaChi,Quan,TinhTrangBDS,DienTich,Phongngu,PhongTam,Loai,Gia,USD,log_price,Rank
0,"Đường Liên Phường, Phường Phú Hữu, Quận 9, Tp ...",Quận 9,Đã bàn giao,50.00,2.0,1.0,Chung cư,1.500000e+09,62500.0,11.042922,62500.0
1,"Lô 5, Lô 5 Khu đô thị Vĩnh Lộc, Phường Bình Hư...",Quận Bình Tân,Đã bàn giao,63.00,2.0,2.0,Chung cư,1.650000e+09,68750.0,11.138232,68750.0
2,"Đường Liên Phường, Phường Phú Hữu, Quận 9, Tp ...",Quận 9,Đã bàn giao,53.00,2.0,1.0,Chung cư,1.600000e+09,66667.0,11.107465,66667.0
3,"Đường Phạm Văn Đồng, Phường Linh Tây, Quận Thủ...",Quận Thủ Đức,Đã bàn giao,75.00,2.0,2.0,Chung cư,1.850000e+09,77083.0,11.252638,77083.0
4,"Số 8, Số 8 Đường Gò Dưa, Phường Hiệp Bình Phướ...",Quận Thủ Đức,Đã bàn giao,57.72,2.0,2.0,Chung cư,1.750000e+09,72917.0,11.197077,72917.0
...,...,...,...,...,...,...,...,...,...,...,...
26063,"Đường Xa lộ Hà Nội, Phường Hiệp Phú, Quận 9, T...",Quận 9,Đã bàn giao,53.00,2.0,1.0,Chung cư,2.100000e+09,87500.0,11.379394,87500.0
26064,"Lô 11B KĐT Nam Thành Phố, Lô 11B KĐT Nam Thành...",Huyện Bình Chánh,Đã bàn giao,76.00,2.0,1.0,Chung cư,1.550000e+09,64583.0,11.075706,64583.0
26065,"Đường Tạ Quang Bửu, Phường 5, Quận 8, Tp Hồ Ch...",Quận 8,Đã bàn giao,115.00,2.0,2.0,Chung cư,3.300000e+09,137500.0,11.831379,137500.0
26066,"Đường Phạm Văn Đồng, Phường 13, Quận Bình Thạn...",Quận Bình Thạnh,Đã bàn giao,82.00,2.0,2.0,Chung cư,2.650000e+09,110417.0,11.612019,110417.0


In [ ]:
df_x = df_dropped_2.iloc[:, 1:9]

df_x

,Quan,TinhTrangBDS,DienTich,Phongngu,PhongTam,Loai,Gia,USD
0,Quận 9,Đã bàn giao,50.00,2.0,1.0,Chung cư,1.500000e+09,62500.0
1,Quận Bình Tân,Đã bàn giao,63.00,2.0,2.0,Chung cư,1.650000e+09,68750.0
2,Quận 9,Đã bàn giao,53.00,2.0,1.0,Chung cư,1.600000e+09,66667.0
3,Quận Thủ Đức,Đã bàn giao,75.00,2.0,2.0,Chung cư,1.850000e+09,77083.0
4,Quận Thủ Đức,Đã bàn giao,57.72,2.0,2.0,Chung cư,1.750000e+09,72917.0
...,...,...,...,...,...,...,...,...
26063,Quận 9,Đã bàn giao,53.00,2.0,1.0,Chung cư,2.100000e+09,87500.0
26064,Huyện Bình Chánh,Đã bàn giao,76.00,2.0,1.0,Chung cư,1.550000e+09,64583.0
26065,Quận 8,Đã bàn giao,115.00,2.0,2.0,Chung cư,3.300000e+09,137500.0
26066,Quận Bình Thạnh,Đã bàn giao,82.00,2.0,2.0,Chung cư,2.650000e+09,110417.0


In [ ]:
## Do giá tiền Việt quá lớn nên chuyển về dạng USD 

df_y = df_dropped_2.iloc[:, 10]
df_y

0         62500.0
1         68750.0
2         66667.0
3         77083.0
4         72917.0
           ...   
26063     87500.0
26064     64583.0
26065    137500.0
26066    110417.0
26067     79167.0
Name: Rank, Length: 26066, dtype: float64

In [ ]:
data_dummies = pd.get_dummies(df_x, drop_first=True)
data_dummies = data_dummies.astype(float)
cols = data_dummies.columns.values
data_preprocessed = data_dummies[cols]
data_preprocessed

,DienTich,Phongngu,PhongTam,Gia,USD,Quan_ Huyện Củ Chi,Quan_ Huyện Hóc Môn,Quan_ Huyện Nhà Bè,Quan_ Quận 1,Quan_ Quận 10,Quan_ Quận 11,Quan_ Quận 12,Quan_ Quận 2,Quan_ Quận 3,Quan_ Quận 4,Quan_ Quận 5,Quan_ Quận 6,Quan_ Quận 7,Quan_ Quận 8,Quan_ Quận 9,Quan_ Quận Bình Thạnh,Quan_ Quận Bình Tân,Quan_ Quận Gò Vấp,Quan_ Quận Phú Nhuận,Quan_ Quận Thủ Đức,Quan_ Quận Tân Bình,Quan_ Quận Tân Phú,TinhTrangBDS_Đã bàn giao,Loai_Căn hộ dịch vụ,"Loai_Căn hộ dịch vụ, mini",Loai_Duplex,Loai_Officetel,Loai_Penthouse,Loai_Tập thể,"Loai_Tập thể, cư xá"
0,50.00,2.0,1.0,1.500000e+09,62500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,63.00,2.0,2.0,1.650000e+09,68750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,53.00,2.0,1.0,1.600000e+09,66667.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,75.00,2.0,2.0,1.850000e+09,77083.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,57.72,2.0,2.0,1.750000e+09,72917.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26063,53.00,2.0,1.0,2.100000e+09,87500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26064,76.00,2.0,1.0,1.550000e+09,64583.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26065,115.00,2.0,2.0,3.300000e+09,137500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26066,82.00,2.0,2.0,2.650000e+09,110417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test = data_preprocessed.iloc[:,0:3]
test

,DienTich,Phongngu,PhongTam
0,50.00,2.0,1.0
1,63.00,2.0,2.0
2,53.00,2.0,1.0
3,75.00,2.0,2.0
4,57.72,2.0,2.0
...,...,...,...
26063,53.00,2.0,1.0
26064,76.00,2.0,1.0
26065,115.00,2.0,2.0
26066,82.00,2.0,2.0


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
#scale data ve dang chuan
scaler = StandardScaler()
scaler.fit(test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(test)

In [ ]:
X_scale

array([[0.09819639, 0.1       , 0.        ],
       [0.1242485 , 0.1       , 0.16666667],
       [0.10420842, 0.1       , 0.        ],
       ...,
       [0.22845691, 0.1       , 0.16666667],
       [0.16232465, 0.1       , 0.16666667],
       [0.11823647, 0.1       , 0.        ]])

In [ ]:
df_y

0         62500.0
1         68750.0
2         66667.0
3         77083.0
4         72917.0
           ...   
26063     87500.0
26064     64583.0
26065    137500.0
26066    110417.0
26067     79167.0
Name: Rank, Length: 26066, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, df_y, test_size=0.3)

In [ ]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(18246, 3) (3910, 3) (3910, 3) (18246,) (3910,) (3910,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(3,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=100,
          validation_data=(X_val, Y_val))

Epoch 1/100
571/571 [==============================] - 15s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/100
571/571 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/100
571/571 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/100
571/571 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/100
571/571 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/100
571/571 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/100
571/571 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accur

In [ ]:
model.evaluate(X_test, Y_test)[1]

86/86 [==============================] - 0s 928us/step - loss: nan - accuracy: 0.0000e+00


0.0